In [1]:
import torch
import torch.nn as nn
import numpy as np
import torch.utils.data as data
import pandas as pd

In [2]:
DATASET_DIR = "C:/Users/db/Desktop/기업-수강ncf/NCF 졸업생 데이터"
train= pd.read_csv(DATASET_DIR+ "/NCF 졸업생 데이터 (1부).csv", sep='|')[['sex','ccd','bzc_cd','grup_cd']]
test = pd.read_csv(DATASET_DIR+ "/NCF 졸업생 데이터 (2부).csv", sep='|')[['sex','ccd','bzc_cd','grup_cd']]

C:\Users\db\AppData\Local\Temp\ipykernel_28680\1374857115.py:2: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  train= pd.read_csv(DATASET_DIR+ "/NCF 졸업생 데이터 (1부).csv", sep='|')[['sex','ccd','bzc_cd','grup_cd']]
C:\Users\db\AppData\Local\Temp\ipykernel_28680\1374857115.py:3: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  test = pd.read_csv(DATASET_DIR+ "/NCF 졸업생 데이터 (2부).csv", sep='|')[['sex','ccd','bzc_cd','grup_cd']]


In [3]:
rating_list = np.concatenate([train.values, test.values])
n_rating_list = []
for a in rating_list:
    sex,ccd,bzc_cd,grup_cd = a
    n_rating_list.append([sex,ccd,bzc_cd,grup_cd])
rating_list = np.array(n_rating_list)

In [4]:
# 모든 유저와 모든 item을 담음
unq_ccd, unq_bzccd,unq_grup_cd = [np.unique(rating_list[:, n+1]) for n in range(3)]
# user와 item의 개수 넣음
num_ccd, num_bzc, num_grup = len(unq_ccd), len(unq_bzccd),len(unq_grup_cd)
print('num_ccd:{} num_bzc:{} num_item:{}'.format(num_ccd, num_bzc, num_grup))
train_list = np.array(train)
test_list = np.array(test)
ccd2idx, bzcidx ,grupidx = {}, {},{}
for i, j in enumerate(unq_ccd):
     ccd2idx[j] = i
for i, j in enumerate(unq_bzccd):
     bzcidx[j] = i
for i, j in enumerate(unq_grup_cd):
     grupidx[j] = i    
from sklearn.model_selection import train_test_split
train_list = [[int(li[0])-1,ccd2idx[str(li[1])], bzcidx[str(li[2])],grupidx[str(li[3])]] for li in rating_list]
# x_train, x_test = train_test_split(train_list, test_size=0.3, shuffle=True, random_state=34)
train_list = pd.DataFrame(train_list, columns = ['sex', 'ccd','bzc','item'])
x_train, x_test = train_test_split(train_list, test_size=0.3, shuffle=True, stratify =train_list[['sex', 'ccd','bzc']], random_state=34)

num_ccd:314 num_bzc:925 num_item:16695


In [5]:
class DfData(torch.utils.data.Dataset):
    def __init__(self,features,num_sex,num_ccd,num_bzc,num_grup,num_ng=0,train_mat=None,is_training=None):
        super(DfData,self).__init__()
        # self.features_ps = torch.Tensor(features).int()
        self.features_ps = features
        self.num_sex = num_sex
        self.num_bzc = num_bzc
        self.num_ccd = num_ccd
        self.num_grup = num_grup
        self.num_ng = num_ng
        self.labels = [0] * len(features)
        self.train_mat = train_mat
        self.is_training = is_training
        
        # negative sampling 
    def set_ng_sample(self):
        assert self.is_training, "no need to sampling when testing"
         
        self.features_ng = []
        for x in self.features_ps:
            # sex
            for _ in range(self.num_ng):
                s = np.random.randint(self.num_sex)
                u = np.random.randint(self.num_ccd)
                q = np.random.randint(self.num_bzc)
                j = np.random.randint(self.num_grup)
                # train set에 있는 경우 다시 뽑기
                self.features_ng.append([s,u,q,j])

        labels_ps = [1] * len(self.features_ps)
        labels_ng = [0] * len(self.features_ng)
        
        self.features_fill = torch.Tensor(self.features_ps + self.features_ng).to(torch.int64)
        self.labels_fill= torch.Tensor(labels_ps + labels_ng)
        

    def __len__(self):
        return (self.num_ng+1) * len(self.labels)
        
    def __getitem__(self,idx):
        features = self.features_fill if self.is_training else torch.Tensor(self.features_ps).to(torch.int64)
        labels = self.labels_fill if self.is_training else torch.Tensor(self.labels)
        
        user = features[idx]
        label = labels[idx]
        return user, label

In [6]:
num_sex = 2
x_train = x_train.values.tolist()
x_test = x_test.values.tolist()
train_dataset = DfData(x_train, num_sex,num_ccd,num_bzc, num_grup, num_ng=1, is_training=True)
train_loader = data.DataLoader(train_dataset, batch_size = 256, shuffle=True, num_workers=0)

In [7]:
train_dataset.set_ng_sample()
for a,b in train_loader:
    print(a)
    print(b)
    break

tensor([[   0,  259,   35, 6500],
        [   1,  192,  494,  130],
        [   0,   20,  753, 6831],
        ...,
        [   0,   95,  330, 4033],
        [   1,  171,   26,  785],
        [   1,   96,  347, 6997]])
tensor([0., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 0., 0., 0., 0., 1., 1.,
        0., 1., 0., 1., 0., 0., 0., 1., 1., 1., 1., 0., 1., 0., 0., 0., 0., 1.,
        1., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 1., 0., 1., 0., 1., 1.,
        0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 1., 1., 0., 0., 0., 0.,
        0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 0., 1., 0., 0.,
        1., 1., 1., 1., 1., 0., 1., 0., 1., 1., 0., 0., 1., 0., 0., 0., 0., 1.,
        1., 0., 0., 0., 1., 1., 1., 0., 1., 0., 0., 0., 1., 1., 1., 0., 1., 0.,
        0., 0., 0., 1., 0., 1., 1., 0., 0., 0., 1., 1., 1., 1., 1., 0., 0., 0.,
        0., 1., 0., 1., 1., 1., 1., 0., 1., 0., 1., 0., 0., 0., 1., 0., 0., 1.,
        0., 1., 1., 1., 1., 0., 1., 0., 0., 0., 0., 1., 0., 0.

In [8]:
class FeaturesEmbedding(torch.nn.Module):

    def __init__(self, field_dims, embed_dim):
        super().__init__()
        self.embedding = torch.nn.Embedding(sum(field_dims), embed_dim)
        self.offsets = np.array((0, *np.cumsum(field_dims)[:-1]), dtype=np.int64())
        torch.nn.init.xavier_uniform_(self.embedding.weight.data)

    def forward(self, x):
        """
        :param x: Long tensor of size ``(batch_size, num_fields)``
        """
        x = x + x.new_tensor(self.offsets).unsqueeze(0)
        return self.embedding(x)

In [9]:
class FeaturesLinear(torch.nn.Module):

    def __init__(self, field_dims, output_dim=1):
        super().__init__()
        self.fc = torch.nn.Embedding(sum(field_dims), output_dim)
        self.bias = torch.nn.Parameter(torch.zeros((output_dim,)))
        self.offsets = np.array((0, *np.cumsum(field_dims)[:-1]), dtype=np.int64())
    def forward(self, x):
        """
        :param x: Long tensor of size ``(batch_size, num_fields)``
        """
        x = x + x.new_tensor(self.offsets).unsqueeze(0)
        return torch.sum(self.fc(x), dim=1) + self.bias


In [10]:
class MultiLayerPerceptron(torch.nn.Module):

    def __init__(self, input_dim, embed_dims, dropout, output_layer=True):
        super().__init__()
        layers = list()
        for embed_dim in embed_dims:
            layers.append(torch.nn.Linear(input_dim, embed_dim))
            layers.append(torch.nn.BatchNorm1d(embed_dim))
            layers.append(torch.nn.ReLU())
            layers.append(torch.nn.Dropout(p=dropout))
            input_dim = embed_dim
        if output_layer:
            layers.append(torch.nn.Linear(input_dim, input_dim))
        self.mlp = torch.nn.Sequential(*layers)

    def forward(self, x):
        """
        :param x: Float tensor of size ``(batch_size, embed_dim)``
        """
        return self.mlp(x)


In [11]:
class CrossNetwork(torch.nn.Module):
    def __init__(self, input_dim):
        super(CrossNetwork, self).__init__()
        self.weights = torch.nn.Parameter(torch.rand(input_dim))

    def forward(self, x):
        # Cross network 연산: x * weights
        return x * self.weights

In [12]:
# import torch
# class DcN(torch.nn.Module):
#     """
#     A pytorch implementation of DeepFM.

#     Reference:
#         H Guo, et al. DeepFM: A Factorization-Machine based Neural Network for CTR Prediction, 2017.
#     """

#     def __init__(self, field_dims, embed_dim, mlp_dims, dropout):
#         super().__init__()
#         self.linear = FeaturesLinear(field_dims)
#         self.embedding = FeaturesEmbedding(field_dims, embed_dim)
#         self.embed_output_dim = len(field_dims) * embed_dim
#         self.mlp = MultiLayerPerceptron(self.embed_output_dim, mlp_dims, dropout)

#     def forward(self, x):
#         """
#         :param x: Long tensor of size ``(batch_size, num_fields)``
#         """
#         embed_x = self.embedding(x)
#         x = self.linear(x) + self.mlp(embed_x.view(-1, self.embed_output_dim))
#         # return torch.sigmoid(x.squeeze(1))
#         return x.squeeze(1)

In [13]:
class DcN(torch.nn.Module):
    def __init__(self, field_dims, embed_dim, mlp_dims, dropout):
        super(DcN, self).__init__()
        # self.linear = FeaturesLinear(field_dims)
        self.embedding = FeaturesEmbedding(field_dims, embed_dim)
        self.embed_output_dim = len(field_dims) * embed_dim
        self.cross_network = CrossNetwork(self.embed_output_dim)
        self.mlp = MultiLayerPerceptron(self.embed_output_dim, mlp_dims, dropout)
        # 출력 레이어
        self.output_layer = torch.nn.Linear(self.embed_output_dim+ embed_dim, 1)

    def forward(self, x):
        # 선형 컴포넌트
        # linear_output = self.linear(x)
        # 임베딩 컴포넌트
        embed_x = self.embedding(x)
        # Cross Network 컴포넌트
        cross_output = self.cross_network(embed_x.view(-1, self.embed_output_dim))
        # MLP 컴포넌트
        mlp_output = self.mlp(embed_x.view(-1, self.embed_output_dim))
        # Cross network 출력과 MLP 출력을 연결
        concat_output = torch.cat([cross_output, mlp_output], dim=1)
        # 최종 출력 레이어
        final_output = self.output_layer(concat_output)
       
        return final_output.squeeze(1)

In [14]:
sex = 2 
field_dims = [sex, num_ccd,num_bzc,num_grup]
embed_dim= 50
mlp_dims = [50, 50] 
dropout= 0.2
device = 'cuda:0'

In [15]:
net = DcN(field_dims, embed_dim , mlp_dims, dropout)
net.to(device)

DcN(
  (embedding): FeaturesEmbedding(
    (embedding): Embedding(17936, 50)
  )
  (cross_network): CrossNetwork()
  (mlp): MultiLayerPerceptron(
    (mlp): Sequential(
      (0): Linear(in_features=200, out_features=50, bias=True)
      (1): BatchNorm1d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): Dropout(p=0.2, inplace=False)
      (4): Linear(in_features=50, out_features=50, bias=True)
      (5): BatchNorm1d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (6): ReLU()
      (7): Dropout(p=0.2, inplace=False)
      (8): Linear(in_features=50, out_features=50, bias=True)
    )
  )
  (output_layer): Linear(in_features=250, out_features=1, bias=True)
)

In [16]:
learning_rate = 0.005
epochs = 50
loss_function = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)

In [17]:
for ep in range(epochs):   
    avg_cost = 0
    train_loader.dataset.set_ng_sample()
    for user, label in train_loader:
        user = user.to(torch.int64).to(device)
        label = label.to('cuda:0')
        net.zero_grad()
        pred = net(user)
        loss = loss_function(pred, label)
        loss.backward()
        optimizer.step()
        avg_cost += loss / len(train_loader)
    net.eval()
    # HR = metrics(net, test_loader, 10)    
    # print(np.mean(HR))
    print('Epoch:', '%04d' % (ep + 1), 'cost =', '{:.9f}'.format(avg_cost)) 

Epoch: 0001 cost = 0.176400155
Epoch: 0002 cost = 0.077050246
Epoch: 0003 cost = 0.055104233
Epoch: 0004 cost = 0.044067748
Epoch: 0005 cost = 0.037714224
Epoch: 0006 cost = 0.033435464
Epoch: 0007 cost = 0.029898429
Epoch: 0008 cost = 0.027933672
Epoch: 0009 cost = 0.025359726
Epoch: 0010 cost = 0.023807231
Epoch: 0011 cost = 0.022574542
Epoch: 0012 cost = 0.021142099
Epoch: 0013 cost = 0.020284399
Epoch: 0014 cost = 0.019497331
Epoch: 0015 cost = 0.018605445
Epoch: 0016 cost = 0.017879575
Epoch: 0017 cost = 0.017124616
Epoch: 0018 cost = 0.016723346
Epoch: 0019 cost = 0.016636958
Epoch: 0020 cost = 0.016084675
Epoch: 0021 cost = 0.015459342
Epoch: 0022 cost = 0.015487282
Epoch: 0023 cost = 0.014316119
Epoch: 0024 cost = 0.014692402
Epoch: 0025 cost = 0.014566072
Epoch: 0026 cost = 0.013851991
Epoch: 0027 cost = 0.014283016
Epoch: 0028 cost = 0.013752769
Epoch: 0029 cost = 0.013403537
Epoch: 0030 cost = 0.013390922
Epoch: 0031 cost = 0.013152560
Epoch: 0032 cost = 0.012623364
Epoch: 0

In [17]:
# torch.save(net,'C:/Users/db/Desktop/DCN/dcn.pth')
model_path = 'C:/Users/db/Desktop/DCN/dcn.pth'

# 모델을 로드합니다.
net = torch.load(model_path)

In [18]:
x_train = np.array(x_train)
x_test = np.array(x_test)

In [19]:
train_dict = {}
for item in x_train:
    key = tuple(item[:3])  # 2, 91, 750을 튜플로 만듭니다.
    value = item[3]  # 나머지 값은 딕셔너리의 값으로 사용합니다.
    
    if key in train_dict:
        train_dict[key].append(value)
    else:
        train_dict[key] = [value]

In [20]:
result_dict = {}
for item in x_test:
    key = tuple(item[:3])  # 2, 91, 750을 튜플로 만듭니다.
    value = item[3]  # 나머지 값은 딕셔너리의 값으로 사용합니다.
    
    if key in result_dict:
        result_dict[key].append(value)
    else:
        result_dict[key] = [value]

In [21]:
keys_list = list(result_dict.keys())
for a in keys_list:
    print(result_dict[a])
    break

[618, 6994, 12453, 509, 14047, 8935, 8942, 8956, 1732, 368, 8987, 8947, 642, 7004, 8963, 8940, 8987, 1076, 11162, 1408, 575, 8953, 8963, 8987, 6998, 8942, 1743, 8966, 1033, 8951, 8946, 1023, 8965, 7004, 8958, 575, 8939, 1016, 1076, 1019, 373, 8961, 8987, 8221, 8987, 380, 6994, 8951, 368, 367, 1668, 374, 8987, 1008, 373, 368, 8949, 8224, 8205, 8934, 7004, 8964, 8966, 1668, 998]


In [22]:
from tqdm import tqdm  # Import tqdm for the progress bar

def precision_and_map(k_values):
    precision_results = {k: [] for k in k_values}
    map_results = {k: [] for k in k_values}
    recall_results = {k: [] for k in k_values}
    
    for i in tqdm(keys_list, desc="Calculating Precision", position=0, leave=True):
        p = []
        try:
            b = result_dict[i]
            c = train_dict[i]
        except KeyError:
            continue
        
        for j in range(num_grup):
            p.append(list(i) + [j])  # Fix variable name from 'a' to 'i'
        
        p = torch.tensor(p).to(device)
        pred = net(p).to(torch.int)  # Change to 'torch.int' for data type
        values, sorted_indices = torch.sort(pred, descending=True)
        
        original_indices = [idx for idx in sorted_indices.tolist() if idx not in set(c)]
        
        for k in k_values:
            exclusive_indices = original_indices[:k]  # Use the original indices here
            # Precision 계산
            intersection_count_k = len(set(b).intersection(set(exclusive_indices)))
            precision_results[k].append(intersection_count_k / k)
            # recall 계산 
            recall_results[k].append(intersection_count_k / len(b))
            # MAP 계산
            precision_at_i = []
            
            for rank, idx in enumerate(exclusive_indices, start=1):
                if idx in set(b):
                    precision_at_i.append(sum([1 if r in set(b) else 0 for r in exclusive_indices[:rank]]) / rank)
            
            if precision_at_i:
                map_results[k].append(sum(precision_at_i) / len(precision_at_i))
            else:
                map_results[k].append(0.0)
    return precision_results, map_results, recall_results

k_values = [10, 25, 50, 100]
precision_results, map_results, recall_results = precision_and_map(k_values)

# Access the results
precision_result_10 = precision_results[10]
precision_result_25 = precision_results[25]
precision_result_50 = precision_results[50]
precision_result_100 = precision_results[100]

map_result_10 = map_results[10]
map_result_25 = map_results[25]
map_result_50 = map_results[50]
map_result_100 = map_results[100]

recall_result_10 = recall_results[10]
recall_result_25 = recall_results[25]
recall_result_50 = recall_results[50]
recall_result_100 = recall_results[100]


Calculating Precision: 100%|███████████████████████████████████████████████████████| 9600/9600 [11:01<00:00, 14.52it/s]


In [23]:
precision_10_mean = np.mean(precision_result_10)
precision_25_mean = np.mean(precision_result_25)
precision_50_mean = np.mean(precision_result_50)
precision_100_mean = np.mean(precision_result_100)

recall_10_mean = np.mean(recall_result_10)
recall_25_mean = np.mean(recall_result_25)
recall_50_mean = np.mean(recall_result_50)
recall_100_mean = np.mean(recall_result_100)

map_10_mean = np.mean(map_result_10)
map_25_mean = np.mean(map_result_25)
map_50_mean = np.mean(map_result_50)
map_100_mean = np.mean(map_result_100)

# 평균 출력
print("Precision@10 Mean:", precision_10_mean)
print("Precision@25 Mean:", precision_25_mean)
print("Precision@50 Mean:", precision_50_mean)
print("Precision@100 Mean:", precision_100_mean)

print("Recall@10 Mean:", recall_10_mean)
print("Recall@25 Mean:", recall_25_mean)
print("Recall@50 Mean:", recall_50_mean)
print("Recall@100 Mean:", recall_100_mean)

print("MAP@10 Mean:", map_10_mean)
print("MAP@25 Mean:", map_25_mean)
print("MAP@50 Mean:", map_50_mean)
print("MAP@100 Mean:", map_100_mean)

Precision@10 Mean: 0.1933125
Precision@25 Mean: 0.15440416666666668
Precision@50 Mean: 0.12157708333333335
Precision@100 Mean: 0.08920729166666667
Recall@10 Mean: 0.10411838123747362
Recall@25 Mean: 0.20802839333683168
Recall@50 Mean: 0.32587206164310706
Recall@100 Mean: 0.47504792649663957
MAP@10 Mean: 0.35573389520738846
MAP@25 Mean: 0.3042322520368684
MAP@50 Mean: 0.2586514440366133
MAP@100 Mean: 0.21824454089997625


In [21]:
from tqdm import tqdm  # Import tqdm for the progress bar

def precision(k):
    result = []
    for i in tqdm(keys_list, desc="Calculating Precision", position=0, leave=True):  # tqdm setup
        p = []
        try:
            b = result_dict[i]
            c = train_dict[i]
        except KeyError:
            continue
        for j in range(num_grup):
            p.append(list(i) + [j])  # Fix variable name from 'a' to 'i'
        
        p = torch.tensor(p).to(device)
        pred = net(p).to(torch.int)  # Change to 'torch.int' for data type
        values, sorted_indices = torch.sort(pred, descending=True)
        
        exclusive_indices = [idx for idx in sorted_indices.tolist() if idx not in set(c)]
        exclusive_indices = exclusive_indices[:k]
        
        intersection_count = len(set(b).intersection(set(exclusive_indices)))
        result.append(intersection_count / k)
    
    return result

In [22]:
from tqdm import tqdm
import torch

def calculate_ap(gt, pred, k):
    """
    단일 클래스에 대한 평균 정밀도 (AP)를 계산합니다.
    """
    precision_at_k = 0.0
    true_positives = 0

    for i in range(k):
        if pred[i] in gt:
            true_positives += 1
            precision_at_k += true_positives / (i + 1)

    if not gt:
        return 0.0

    return precision_at_k / len(gt)

def mean_average_precision(k):
    result = []

    for i in tqdm(keys_list, desc="mAP 계산 중", position=0, leave=True):
        try:
            b = result_dict[i]
            c = train_dict[i]
        except KeyError:
            continue

        p = []
        for j in range(num_grup):
            p.append(list(i) + [j])
        p = torch.tensor(p).to(device)
        pred = net(p).to(torch.int)
        values, sorted_indices = torch.sort(pred, descending=True)

        exclusive_indices = [idx for idx in sorted_indices.tolist() if idx not in set(c)] 
        exclusive_indices = exclusive_indices[:k]

        ap = calculate_ap(b, exclusive_indices, k)
        result.append(ap)

    mean_ap = sum(result) / len(result) if result else 0.0
    return mean_ap


In [23]:
map_10 = mean_average_precision(10)
map_25 = mean_average_precision(25)
map_50 = mean_average_precision(50)
map_100 = mean_average_precision(100)

mAP 계산 중: 100%|█████████████████████████████████████████████████████████████████| 9600/9600 [08:03<00:00, 19.87it/s]


In [24]:
print(map_10)
print(map_25)
print(map_50)
print(map_100)

0.05348804535284291
0.07933083314491068
0.09900535963675014
0.11617460278865101


In [23]:
# Example usage:
a_10 = precision(10)
a_25 = precision(25)
a_50 = precision(50)
a_100 = precision(100)

Calculating Precision: 100%|███████████████████████████████████████████████████████| 9600/9600 [09:53<00:00, 16.17it/s]


In [24]:
print(np.array(a_10).mean())
print(np.array(a_25).mean())
print(np.array(a_50).mean())
print(np.array(a_100).mean())

0.1933125
0.15440416666666668
0.12157708333333335
0.08920729166666667


In [25]:
from tqdm import tqdm  # Import tqdm for the progress bar

def Recall(k):
    result = []
    for i in tqdm(keys_list, desc="Calculating Precision", position=0, leave=True):  # tqdm setup
        p = []
        try:
            b = result_dict[i]
            c = train_dict[i]
        except KeyError:
            continue
        for j in range(num_grup):
            p.append(list(i) + [j])  # Fix variable name from 'a' to 'i'
        
        p = torch.tensor(p).to(device)
        pred = net(p).to(torch.int)  # Change to 'torch.int' for data type
        values, sorted_indices = torch.sort(pred, descending=True)
        
        exclusive_indices = [idx for idx in sorted_indices.tolist() if idx not in set(c)]
        exclusive_indices = exclusive_indices[:k]
        
        intersection_count = len(set(b).intersection(set(exclusive_indices)))
        result.append(intersection_count / len(set(b)))
    
    return result

In [26]:
b_10 = Recall(10)
b_25 = Recall(25)
b_50 = Recall(50)
b_100 = Recall(100)

Calculating Precision: 100%|███████████████████████████████████████████████████████| 9600/9600 [10:56<00:00, 14.63it/s]


In [27]:
print(np.array(b_10).mean())
print(np.array(b_25).mean())
print(np.array(b_50).mean())
print(np.array(b_100).mean())

0.10769953555506812
0.214977293497301
0.33680489581424716
0.49104891730649625
